In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [3]:
gold = pd.read_csv('/workspaces/setproject/set1.csv')

In [25]:
features = ["Open", "High", "Low"]
target="Price"

In [26]:
def categorize_movement(price_change):
    if price_change > 0:
        return "Up"
    elif price_change < 0:
        return "Down"
    else:
        return "No Change"  # Handle cases where price remains the same

gold["Movement"] = gold[target].pct_change().apply(categorize_movement)

In [6]:
threshold = 0  # Adjust threshold as needed (e.g., 0.5% change)
gold_data["Up"] = (gold_data["Change"] > threshold).astype(int)

/tmp/ipykernel_10556/3920141983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gold_data["Up"] = (gold_data["Change"] > threshold).astype(int)


In [28]:
X = gold[features]
y = gold["Movement"]

In [29]:
# Standardize features for better SVM performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [30]:
selector = SelectKBest(f_classif, k=2)
X_selected = selector.fit_transform(X_scaled, y)

In [31]:
print(selector.scores_)
print(selector.get_support())

[1.96982064 1.9430627  1.84094972]
[ True  True False]


In [56]:
pca = PCA(n_components=1)
X_reduced = pca.fit_transform(X_selected)

In [64]:
# Print or visualize the data with reduced dimensions (optional)
print(pd.DataFrame(X_reduced, columns=["PC1"]))

           PC1
0     2.089493
1     2.049508
2     2.064991
3     2.078018
4     1.989979
...        ...
2578  1.159221
2579  1.178420
2580  1.199555
2581  1.332201
2582  1.314220

[2583 rows x 1 columns]


In [65]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100)  # Adjust n_estimators (number of trees)
rf.fit(X_reduced, y)

RandomForestClassifier()

In [66]:
# Split data into training and testing sets (consider cross-validation for more robust evaluation)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2)

# Evaluate Random Forest performance
rf_accuracy = accuracy_score(y_test, rf.predict(X_test))
rf_f1 = f1_score(y_test, rf.predict(X_test), average='weighted')

# Print or compare performance metrics
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest F1-score:", rf_f1)

Random Forest Accuracy: 0.9903288201160542
Random Forest F1-score: 0.9885013234600135


In [67]:
from sklearn.svm import SVC  # Simulated QSVM

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2)

# Train Simulated QSVM (using scikit-learn SVM with RBF kernel)
qsvm = SVC(kernel='rbf', random_state=100)
qsvm.fit(X_train, y_train)

SVC(random_state=100)

In [69]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100)  # Adjust n_estimators (number of trees)
rf.fit(X_train, y_train)

# Evaluate performance on testing data
qsvm_accuracy = accuracy_score(y_test, qsvm.predict(X_test))
qsvm_f1 = f1_score(y_test, qsvm.predict(X_test),average='weighted')

In [70]:
rf_accuracy = accuracy_score(y_test, rf.predict(X_test))
rf_f1 = f1_score(y_test, rf.predict(X_test),average='weighted')

# Print and compare performance metrics
print("QSVM Accuracy:", qsvm_accuracy)
print("QSVM F1-score:", qsvm_f1)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest F1-score:", rf_f1)

QSVM Accuracy: 0.47775628626692457
QSVM F1-score: 0.44258249517404946
Random Forest Accuracy: 0.48549323017408125
Random Forest F1-score: 0.48559796805424815
